## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering


## OLD STUFF IGNORE

In [2]:
df = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')
df

C:\Users\sathi\AppData\Local\Temp\ipykernel_16936\870708999.py:1: DtypeWarning: Columns (1,3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.1,18247,149.171.126.4,7662,tcp,FIN,0.119596,4550,68342,31,...,,6,2,2,5,1,1,2,NaN,0
1,59.166.0.3,54771,149.171.126.2,27709,tcp,FIN,0.650574,8928,320,31,...,,3,5,2,4,1,1,4,NaN,0
2,59.166.0.8,13289,149.171.126.9,5190,tcp,FIN,0.007980,2158,2464,31,...,,3,5,1,1,1,1,3,NaN,0
3,149.171.126.18,1043,175.45.176.3,53,udp,INT,0.000005,264,0,60,...,,19,19,19,19,19,19,19,NaN,0
4,149.171.126.18,1043,175.45.176.3,53,udp,INT,0.000005,264,0,60,...,,19,19,19,19,19,19,19,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540042,59.166.0.5,33094,149.171.126.7,43433,tcp,FIN,0.087306,320,1828,31,...,,1,2,3,3,1,1,3,NaN,0
2540043,59.166.0.7,20848,149.171.126.4,21,tcp,CON,0.365058,456,346,31,...,2,2,2,2,2,2,2,2,NaN,0
2540044,59.166.0.3,21511,149.171.126.9,21,tcp,CON,6.335154,1802,2088,31,...,2,2,2,4,2,2,2,2,NaN,0
2540045,59.166.0.9,35433,149.171.126.0,80,tcp,CON,2.200934,3498,166054,31,...,,1,1,2,4,2,2,2,NaN,0


 Cleaning dataset

Drop is_sm_ips_ports

In [3]:
#Cleaning df by removing rows with is_sm_ips_ports = 1
df = df[df['is_sm_ips_ports']==0]
#Dropping is_sm_ips_ports column
df.drop(['is_sm_ips_ports'], axis=1, inplace=True)

C:\Users\sathi\AppData\Local\Temp\ipykernel_16936\1444029592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['is_sm_ips_ports'], axis=1, inplace=True)


In [4]:
#Find out unique values in df['attack_cat']
df['attack_cat'].unique()

array([nan, 'Exploits', ' Fuzzers ', 'DoS', 'Generic', ' Shellcode ',
       'Backdoor', ' Reconnaissance ', 'Worms', 'Analysis',
       'Reconnaissance', 'Shellcode', ' Fuzzers', 'Backdoors'],
      dtype=object)

In [5]:
#Remove trailing white spaces from the dataset
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [6]:
df['attack_cat'].unique()

array([nan, 'Exploits', 'Fuzzers', 'DoS', 'Generic', 'Shellcode',
       'Backdoor', 'Reconnaissance', 'Worms', 'Analysis', 'Backdoors'],
      dtype=object)

In [7]:
#Replace 'Backdoor' with 'Backdoors'
df['attack_cat'] = df['attack_cat'].replace('Backdoor', 'Backdoors')

In [8]:
#Fill NaN values with 0 for int and float columns and None for object columns
for col in df.columns:
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        df[col] = df[col].fillna(0)
    elif df[col].dtype == 'object':
        df[col] = df[col].fillna('None')

In [9]:
df['attack_cat'].unique()

array(['None', 'Exploits', 'Fuzzers', 'DoS', 'Generic', 'Shellcode',
       'Backdoors', 'Reconnaissance', 'Worms', 'Analysis'], dtype=object)

In [10]:
#Replace None in sport and dport with 0
df['sport'] = df['sport'].replace('None', '0')
df['dsport'] = df['dsport'].replace('None', '0')

In [11]:
df.is_ftp_login.unique()

array([0., 1., 4., 2.])

In [12]:
#Convert is_ftp_login to int
df['is_ftp_login'] = df['is_ftp_login'].astype(int)

In [13]:
#Convert all is_ftp_login values > 0 to 1
df['is_ftp_login'] = df['is_ftp_login'].apply(lambda x: 1 if x > 0 else x)

In [14]:
df.ct_dst_src_ltm .unique()

array([ 2,  4,  3, 19,  1,  5,  7,  6, 25, 24, 10,  9, 18, 22, 17, 32, 54,
       15, 33, 48, 40, 37, 16, 42, 34, 39, 31, 36, 11, 30, 21, 27, 29, 35,
       43, 59, 23, 47, 20, 14, 38, 56, 45,  8, 51, 26, 41, 28, 49, 12, 46,
       13, 44, 53, 50, 52, 57, 58, 55, 60, 66, 61, 65, 67, 63],
      dtype=int64)

In [15]:
#Convert to csv and store in a new file
df.to_csv(r'D:\Project Phase II\Dataset\phase2cleaneddataset.csv', index=False)

In [16]:
df.drop(['attack_cat'],axis=1,inplace=True)

Label Encoding

In [17]:
#label encoding for non numerical features
labelencoder = LabelEncoder()
from sklearn import preprocessing 

# label_encoder object knows  
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

df['proto'] = labelencoder.fit_transform(df['proto'])
df['service'] = labelencoder.fit_transform(df['service'])
df['state'] = labelencoder.fit_transform(df['state'])
df['attack_cat'] = labelencoder.fit_transform(df['attack_cat'])



KeyError: 'attack_cat'

In [ ]:
df2 = pd.read_csv(r'D:\Project Phase II\Dataset\phase2cleaneddataset.csv',encoding='cp1252')
df2

C:\Users\sathi\AppData\Local\Temp\ipykernel_20212\128579614.py:1: DtypeWarning: Columns (1,3,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(r'D:\Project Phase II\Dataset\phase2cleaneddataset.csv',encoding='cp1252')


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.1,0,149.171.126.4,7662,tcp,FIN,0.119596,4550,68342,31,...,NaN,6,2,2,5,1,1,2,None,0
1,59.166.0.3,0,149.171.126.2,27709,tcp,FIN,0.650574,8928,320,31,...,NaN,3,5,2,4,1,1,4,None,0
2,59.166.0.8,0,149.171.126.9,5190,tcp,FIN,0.007980,2158,2464,31,...,NaN,3,5,1,1,1,1,3,None,0
3,149.171.126.18,0,175.45.176.3,53,udp,INT,0.000005,264,0,60,...,NaN,19,19,19,19,19,19,19,None,0
4,149.171.126.18,0,175.45.176.3,53,udp,INT,0.000005,264,0,60,...,NaN,19,19,19,19,19,19,19,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535847,59.166.0.5,0,149.171.126.7,0,tcp,FIN,0.087306,320,1828,31,...,NaN,1,2,3,3,1,1,3,None,0
2535848,59.166.0.7,0,149.171.126.4,0,tcp,CON,0.365058,456,346,31,...,2.0,2,2,2,2,2,2,2,None,0
2535849,59.166.0.3,0,149.171.126.9,0,tcp,CON,6.335154,1802,2088,31,...,2.0,2,2,4,2,2,2,2,None,0
2535850,59.166.0.9,0,149.171.126.0,0,tcp,CON,2.200934,3498,166054,31,...,NaN,1,1,2,4,2,2,2,None,0


In [ ]:

#Replace '' and 'None' with 0 in ct_ftp_cmd
df2['ct_ftp_cmd'] = df2['ct_ftp_cmd'].replace('None', '0')
df2['ct_ftp_cmd'] = df2['ct_ftp_cmd'].replace('', '0')

In [ ]:
df2.ct_ftp_cmd.fillna(0, inplace=True)

In [ ]:
df2['ct_ftp_cmd'] = df2['ct_ftp_cmd'].astype(int)

In [ ]:
df2['ct_ftp_cmd'].unique()

array([0, 1, 4, 2, 6])

In [ ]:
#Convert to csv and store in a new file
df2.to_csv(r'D:\Project Phase II\Dataset\phase2cleaneddataset.csv', index=False)

In [ ]:
#convert ct_ftp_cmd to int
df['ct_ftp_cmd'] = df['ct_ftp_cmd'].astype(int)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2535852 entries, 0 to 2540046
Data columns (total 48 columns):
 #   Column            Dtype  
---  ------            -----  
 0   srcip             object 
 1   sport             object 
 2   dstip             object 
 3   dsport            object 
 4   proto             int32  
 5   state             int32  
 6   dur               float64
 7   sbytes            int64  
 8   dbytes            int64  
 9   sttl              int64  
 10  dttl              int64  
 11  sloss             int64  
 12  dloss             int64  
 13  service           int32  
 14  Sload             float64
 15  Dload             float64
 16  Spkts             int64  
 17  Dpkts             int64  
 18  swin              int64  
 19  dwin              int64  
 20  stcpb             int64  
 21  dtcpb             int64  
 22  smeansz           int64  
 23  dmeansz           int64  
 24  trans_depth       int64  
 25  res_bdy_len       int64  
 26  Sjit          

In [ ]:
#Convert to csv and store in a new file
df.to_csv(r'D:\Project Phase II\Dataset\phase2labelencodeddataset.csv', index=False)

Filling in null values

In [ ]:
#Fill nullvalues
#Filling Nan values with 0
df.fillna(0, inplace=True)
#Filling '' with 0
df.replace('', 0, inplace=True)
df.replace(' ', 0, inplace=True)

C:\Users\sathi\AppData\Local\Temp\ipykernel_2968\201358963.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)
C:\Users\sathi\AppData\Local\Temp\ipykernel_2968\201358963.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace('', 0, inplace=True)
C:\Users\sathi\AppData\Local\Temp\ipykernel_2968\201358963.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(' ', 0, inplace=True)


In [ ]:
df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,59.166.0.1,18247,149.171.126.4,7662,114,5,0.119596,4550,68342,31,...,0.0,0,6,2,2,5,1,1,2,0
1,59.166.0.3,54771,149.171.126.2,27709,114,5,0.650574,8928,320,31,...,0.0,0,3,5,2,4,1,1,4,0
2,59.166.0.8,13289,149.171.126.9,5190,114,5,0.007980,2158,2464,31,...,0.0,0,3,5,1,1,1,1,3,0
3,149.171.126.18,1043,175.45.176.3,53,120,6,0.000005,264,0,60,...,0.0,0,19,19,19,19,19,19,19,0
4,149.171.126.18,1043,175.45.176.3,53,120,6,0.000005,264,0,60,...,0.0,0,19,19,19,19,19,19,19,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540042,59.166.0.5,33094,149.171.126.7,43433,114,5,0.087306,320,1828,31,...,0.0,0,1,2,3,3,1,1,3,0
2540043,59.166.0.7,20848,149.171.126.4,21,114,2,0.365058,456,346,31,...,2.0,2,2,2,2,2,2,2,2,0
2540044,59.166.0.3,21511,149.171.126.9,21,114,2,6.335154,1802,2088,31,...,2.0,2,2,2,4,2,2,2,2,0
2540045,59.166.0.9,35433,149.171.126.0,80,114,2,2.200934,3498,166054,31,...,0.0,0,1,1,2,4,2,2,2,0


Correlation with label

In [ ]:
#Correlation with output variable
cor = df.corr()
cor_target = abs(cor["Label"])
cor_target

proto               0.011832
state               0.338312
dur                 0.002055
sbytes              0.010134
dbytes              0.075698
sttl                0.904512
dttl                0.134569
sloss               0.043423
dloss               0.096054
service             0.009492
Sload               0.192121
Dload               0.220163
Spkts               0.121179
Dpkts               0.116195
swin                0.316271
dwin                0.315079
stcpb               0.234160
dtcpb               0.234252
smeansz             0.065710
dmeansz             0.273102
trans_depth         0.029195
res_bdy_len         0.027398
Sjit                0.020903
Djit                0.054562
Stime               0.275647
Ltime               0.275647
Sintpkt             0.012086
Dintpkt             0.010668
tcprtt              0.143044
synack              0.122190
ackdat              0.143386
ct_state_ttl        0.880216
ct_flw_http_mthd    0.026690
is_ftp_login        0.031908
ct_srv_src    

In [ ]:
sorted(cor_target,reverse=True)

[1.0,
 0.9045123713273255,
 0.8802159911646181,
 0.4397582040181459,
 0.41894609595431975,
 0.3966171141155974,
 0.3862785180682943,
 0.38276987066060003,
 0.34300251941699716,
 0.33923406199653117,
 0.3383115509958634,
 0.31627126136549794,
 0.31507938791330836,
 0.2756467745810105,
 0.2756467361934781,
 0.2731022353548177,
 0.23425165967570538,
 0.23415972437915167,
 0.22016301877421282,
 0.19212078068924934,
 0.14338606748651497,
 0.14304379120018665,
 0.1345687229012209,
 0.12218956387082323,
 0.12117868665702373,
 0.11619454701110364,
 0.09605449315010073,
 0.07569769679289255,
 0.06571002635878419,
 0.054562098246944746,
 0.043423355305197676,
 0.03190817000838083,
 0.02919529475181047,
 0.02739821746445534,
 0.02668985508780704,
 0.020902958909795925,
 0.01208573079157584,
 0.011832106432080278,
 0.010667773256950744,
 0.010133696212686666,
 0.009492262382872188,
 0.002055001885478537]

In [ ]:
#Information Gain of features
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

X = df.iloc[:,0:42]  #independent columns
y = df.iloc[:,-1]    #target column i.e Label


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

In [ ]:
#SAmple the dataset
# Randomly sample 1000 rows
dfsample = df.sample(n=1000, random_state=42)

# Extract features and labels
src_encoder = LabelEncoder()
x = src_encoder.fit_transform(x)
dst_encoder = LabelEncoder()
y = dst_encoder.fit_transform(data['Dst IP Addr'])

x = torch.tensor(dfsample[['srcip', 'dstip']].values, dtype=torch.long)
y = torch.tensor(dfsample['Label'].values, dtype=torch.long)

features = dfsample.drop(columns=['srcip', 'dstip','sport','dsport', 'Label'])
scaler = StandardScaler()
features = scaler.fit_transform(features)

#Sampling
# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Construct edge indices
edge_index = torch.tensor([dfsample['srcip'].values, dfsample['dstip'].values], dtype=torch.long)
edge_attr = torch.tensor(features, dtype=torch.ListType) 

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        return x

# Instantiate the model
model = GCN(input_dim=2, hidden_dim=64, output_dim=2)  # input_dim=2 since we have 2 features (source and destination IPs), output_dim=2 for binary edge classification

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train_model(model, optimizer, criterion, x_train, edge_index, edge_attr, y_train, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(x_train, edge_index, edge_attr)
        loss = criterion(out, y_train)
        loss.backward()
        optimizer.step()
        print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

# Convert edge labels to one-hot encoding
y_train_onehot = F.one_hot(y_train)

# Train the model
train_model(model, optimizer, criterion, x_train, edge_index, edge_attr, y_train, epochs=10)

TypeError: tensor(): argument 'dtype' must be torch.dtype, not pybind11_type

In [ ]:
# Testing
def test_model(model, x_test, edge_index, edge_attr, y_test):
    model.eval()
    with torch.no_grad():
        out = model(x_test, edge_index, edge_attr)
        pred = out.argmax(dim=1)
        f1 = f1_score(y_test, pred, average='macro')
        print(f'F1 Score: {f1}')

# Test the model
test_model(model, x_test, edge_index, edge_attr, y_test)

## Import Dataset

In [2]:
df = pd.read_csv(r'D:\Project Phase II\Dataset\phase2labelencodeddataset.csv',encoding='cp1252')

C:\Users\sathi\AppData\Local\Temp\ipykernel_6092\3370251438.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\Project Phase II\Dataset\phase2labelencodeddataset.csv',encoding='cp1252')


In [3]:
df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.1,0,149.171.126.4,7662,114,5,0.119596,4550,68342,31,...,0,6,2,2,5,1,1,2,6,0
1,59.166.0.3,0,149.171.126.2,27709,114,5,0.650574,8928,320,31,...,0,3,5,2,4,1,1,4,6,0
2,59.166.0.8,0,149.171.126.9,5190,114,5,0.007980,2158,2464,31,...,0,3,5,1,1,1,1,3,6,0
3,149.171.126.18,0,175.45.176.3,53,120,6,0.000005,264,0,60,...,0,19,19,19,19,19,19,19,6,0
4,149.171.126.18,0,175.45.176.3,53,120,6,0.000005,264,0,60,...,0,19,19,19,19,19,19,19,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535847,59.166.0.5,0,149.171.126.7,0,114,5,0.087306,320,1828,31,...,0,1,2,3,3,1,1,3,6,0
2535848,59.166.0.7,0,149.171.126.4,0,114,2,0.365058,456,346,31,...,2,2,2,2,2,2,2,2,6,0
2535849,59.166.0.3,0,149.171.126.9,0,114,2,6.335154,1802,2088,31,...,2,2,2,4,2,2,2,2,6,0
2535850,59.166.0.9,0,149.171.126.0,0,114,2,2.200934,3498,166054,31,...,0,1,1,2,4,2,2,2,6,0


## EDA

In [4]:
nodes = set()
for i in range(len(df)):
    nodes.add(str(df['srcip'][i])+':'+str(df['sport'][i]))
    nodes.add(str(df['srcip'][i])+':'+str(df['sport'][i]))

In [5]:
len(nodes)

68192

## Baseline ML Models

In [7]:
#Use logistic regression to predict the label
#Split the dataset into features and labels
X = df['sttl']
y = df['Label']

#Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Logistic regression to classify only using one variable
LR = LogisticRegression()
LR.fit(X_train.values.reshape(-1,1), y_train)

y_pred = LR.predict(X_test.values.reshape(-1,1))



In [9]:
#Confusion matrix
confusion_matrix(y_test, y_pred)


array([[436361,   6577],
       [  4599,  59634]], dtype=int64)

In [11]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    442938
           1       0.90      0.93      0.91     64233

    accuracy                           0.98    507171
   macro avg       0.95      0.96      0.95    507171
weighted avg       0.98      0.98      0.98    507171



In [8]:
#F1 score for each label
f1_score(y_test, y_pred, average=None)

array([0.98735601, 0.91432339])

In [12]:
#PCA for dimensionality reduction using only two variables sttl and ct_state_ttl
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X = df[['sttl', 'ct_state_ttl']]
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pca_x_train = pca.fit_transform(X_train)
pca_x_test = pca.fit_transform(X_test)
classifier = LogisticRegression()
classifier.fit(pca_x_train, y_train)
y_pred = classifier.predict(pca_x_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99    442938
           1       0.90      0.93      0.91     64233

    accuracy                           0.98    507171
   macro avg       0.95      0.96      0.95    507171
weighted avg       0.98      0.98      0.98    507171



In [13]:
#Linear equation formed by PCA
print(pca.components_)

[[0.99996518 0.00834444]]


## Find Prediction Values of Baseline ML Models Alone

In [3]:
#Find all string values in sport
df['sport'] = df['sport'].astype(str)
#Replace '-' with 0
df['sport'] = df['sport'].replace('-', '80')
#COnvert hexadecimal values to decimal in sport
df['sport'] = df['sport'].apply(lambda x: int(x, 16))
#Convert sport to int
df['sport'] = df['sport'].astype(int)

len(df['sport'].unique())

43005

In [4]:
df['dsport'] = df['dsport'].astype(str)
df['dsport'] = df['dsport'].replace('-', '80')
df['dsport'] = df['dsport'].apply(lambda x: int(x, 16))
df['dsport'] = df['dsport'].astype(int)
len(df['dsport'].unique())


64380

In [5]:
dfSample0 = df[df['Label'] == 0].sample(n=250000, random_state=42)
dfSample1 = df[df['Label'] == 1].sample(n=250000, random_state=42)

dfSample = pd.concat([dfSample0, dfSample1])
dfSample = dfSample.sample(frac=1, random_state=42).reset_index(drop=True)

#Split the dataset into features and labels
XSample = dfSample.drop('Label', axis=1)
ySample = dfSample['Label']

#Split the dataset into training and testing sets
XSample_train, XSample_test, ySample_train, ySample_test = train_test_split(XSample, ySample, test_size=0.2, random_state=42)


In [6]:
#Take equal number of samples for each label
dfDroppedforLR = df[df['Label'] == 0].sample(n=250000, random_state=40)
dfDroppedforLR = dfDroppedforLR.append(df[df['Label'] == 1].sample(n=250000, random_state=40))

#Drop srcip and dstip and use a new df
dfDroppedforLR = dfDroppedforLR.drop(['srcip', 'dstip','attack_cat','Stime','Ltime'], axis=1)

#Shuffle the dataset
dfDroppedforLR = dfDroppedforLR.sample(frac=1).reset_index(drop=True)

#Split the dataset into features and labels
X = dfDroppedforLR.iloc[:,0:42]
y = dfDroppedforLR['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


C:\Users\sathi\AppData\Local\Temp\ipykernel_6092\3350324575.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfDroppedforLR = dfDroppedforLR.append(df[df['Label'] == 1].sample(n=250000, random_state=40))


Logistic Regression

In [40]:
#Logistic regression to classify using all variables
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.87      0.89     49914
           1       0.88      0.92      0.90     50086

    accuracy                           0.90    100000
   macro avg       0.90      0.90      0.90    100000
weighted avg       0.90      0.90      0.90    100000



SVC

In [ ]:
#SVC
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(classification_report(y_test,y_pred))

Lazy Predict

In [23]:
#Use lazypredict 
from lazypredict.Supervised import LazyClassifier
#Remove time consuming lazypredict classifier models from training


In [11]:
#list all classifier names in lazypredict and remove unwanted ones
LazyClassifier().models


{}

In [26]:
# List of classifiers to include
import sklearn
import xgboost
import lightgbm

classifiers = [
 ('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('CategoricalNB', sklearn.naive_bayes.CategoricalNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('KNeighborsClassifier',  sklearn.neighbors._classification.KNeighborsClassifier),
 ('LinearDiscriminantAnalysis',  sklearn.discriminant_analysis.LinearDiscriminantAnalysis),
 ('LinearSVC', sklearn.svm._classes.LinearSVC),
 ('LogisticRegression', sklearn.linear_model._logistic.LogisticRegression),
 ('NuSVC', sklearn.svm._classes.NuSVC),
 ('PassiveAggressiveClassifier',  sklearn.linear_model._passive_aggressive.PassiveAggressiveClassifier),
 ('Perceptron', sklearn.linear_model._perceptron.Perceptron),
 ('QuadraticDiscriminantAnalysis',  sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis),
 ('RandomForestClassifier', sklearn.ensemble._forest.RandomForestClassifier),
 ('RidgeClassifier', sklearn.linear_model._ridge.RidgeClassifier),
 ('RidgeClassifierCV', sklearn.linear_model._ridge.RidgeClassifierCV),
 ('SGDClassifier', sklearn.linear_model._stochastic_gradient.SGDClassifier),
 ('SVC', sklearn.svm._classes.SVC),
 ('StackingClassifier', sklearn.ensemble._stacking.StackingClassifier),
 ('XGBClassifier', xgboost.sklearn.XGBClassifier),
 ('LGBMClassifier', lightgbm.sklearn.LGBMClassifier)]
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None,classifiers=classifiers)


In [22]:
from lazypredict import Supervised
Supervised.CLASSIFIERS

[('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('CalibratedClassifierCV', sklearn.calibration.CalibratedClassifierCV),
 ('CategoricalNB', sklearn.naive_bayes.CategoricalNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('ExtraTreeClassifier', sklearn.tree._classes.ExtraTreeClassifier),
 ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('KNeighborsClassifier',
  sklearn.neighbors._classification.KNeighborsClassifier),
 ('LabelPropagation',
  sklearn.semi_supervised._label_propagation.LabelPropagation),
 ('LabelSpreading', sklearn.semi_supervised._label_propagation.LabelSpreading),
 ('LinearDiscriminantAnalysis',
  sklearn.discriminant_analysis.LinearDiscriminantAnalysis),
 ('Linear

In [27]:
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


 48%|████▊     | 11/23 [35:49<45:40, 228.36s/it]  

In [13]:
#Classification report for lazypredict models
for model in models:
    print(model)
    print(classification_report(y_test, predictions[model]))
    print('-----------------------------------------------------------------------------------')

NameError: name 'models' is not defined

## LSTM 

1. LSTM FOR AVG STTL

Sample the dataset (1% sample taken)

In [5]:
# Prepare data (sample data)
# Replace this part with your actual data loading and preprocessing

#Sample 1% of the dataset
dfsample = df.sample(frac=0.01, random_state=42)

nodes = set()
for i in range(len(dfsample)):
    src = str(dfsample['srcip'].iloc[i])+':'+str(dfsample['sport'].iloc[i])
    dst = str(dfsample['dstip'].iloc[i])+':'+str(dfsample['dsport'].iloc[i])
    nodes.add(src)
    nodes.add(dst)

In [6]:
len(nodes)

5325

In [7]:
len(dfsample)

25359

Model Training


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Modify the call method of the LSTMClassifier
class LSTMClassifier(tf.keras.Model):
    def __init__(self, input_shape, hidden_units, output_units):
        super(LSTMClassifier, self).__init__()
        self.lstm = LSTM(hidden_units, return_sequences=False)  # Set return_sequences=False since we only need the output at the last timestep
        self.dense = Dense(output_units, activation='sigmoid')

    def call(self, inputs):
        x = tf.expand_dims(inputs, axis=-1)  # Add a new dimension to represent the input features
        x = self.lstm(x)
        output = self.dense(x)
        return output

# Prepare an adjacency matrix of the nodes
adjacency_matrix = np.zeros((len(nodes), len(nodes)), dtype=float)
count_matrix = np.zeros((len(nodes), len(nodes)), dtype=int)
label_matrix = np.zeros((len(nodes), len(nodes)))

# Add the columns srcip:sport and dstip:dsport to the adjacency matrix
for i in range(len(dfsample)):
    src = str(dfsample['srcip'].iloc[i])+':'+str(dfsample['sport'].iloc[i])
    dst = str(dfsample['dstip'].iloc[i])+':'+str(dfsample['dsport'].iloc[i])
    src_index = list(nodes).index(src)
    dst_index = list(nodes).index(dst)
    if dfsample['Label'].iloc[i] == 1:
        if label_matrix[src_index,dst_index] == 0:
            adjacency_matrix[src_index, dst_index] = dfsample['sttl'].iloc[i]
            label_matrix[src_index, dst_index] = 1  
            count_matrix[src_index,dst_index]+= 1
        else:
            adjacency_matrix[src_index, dst_index] = (adjacency_matrix[src_index, dst_index]*count_matrix[src_index,dst_index]+dfsample['sttl'].iloc[i])/(count_matrix[src_index,dst_index]+1)
            count_matrix[src_index,dst_index]+= 1
    else:
        if count_matrix[src_index,dst_index] == 0:
            adjacency_matrix[src_index, dst_index] = dfsample['sttl'].iloc[i]
            count_matrix[src_index,dst_index]+= 1
        else:
            adjacency_matrix[src_index, dst_index] = (adjacency_matrix[src_index, dst_index]*count_matrix[src_index,dst_index]+dfsample['sttl'].iloc[i])/(count_matrix[src_index,dst_index]+1)
            count_matrix[src_index,dst_index]+= 1

# Flatten tuple values in the adjacency matrix
adjacency_matrix_flat = adjacency_matrix.reshape(-1, 1)
label_flat = label_matrix.reshape(-1, 1)

# Hyperparameters
input_shape = adjacency_matrix_flat.shape[1:]  # Assuming each tuple has 2 attributes
hidden_units = 16
output_units = 1  # Binary classification

# Convert adjacency_matrix_flat to a TensorFlow tensor
adjacency_tensor = tf.convert_to_tensor(adjacency_matrix_flat, dtype=tf.float32)

# Initialize model, optimizer, and loss function
model = LSTMClassifier(input_shape, hidden_units, output_units)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



# Training
history = model.fit(adjacency_tensor, label_flat, epochs=2, batch_size=32, validation_split=0.2)

# Evaluation
#loss, accuracy = model.evaluate(adjacency_tensor, label_flat) 
#print(f'Loss: {loss}, Accuracy: {accuracy}')





Epoch 1/2


708891/708891 [==============================] - 5248s 7ms/step - loss: 2.7351e-04 - accuracy: 1.0000 - val_loss: 2.3519e-05 - val_accuracy: 1.0000
Epoch 2/2
 14804/886114 [..............................] - ETA: 1:04:36 - loss: 1.2331e-06 - accuracy: 1.0000

KeyboardInterrupt: 

Preparting Testing data 

In [21]:
#Take a sample from the dataset to evalute the lstm model
dftestsample = df.sample(frac=0.001, random_state=32)

nodestest = set()
for i in range(len(dftestsample)):
    src = str(dftestsample['srcip'].iloc[i])+':'+str(dftestsample['sport'].iloc[i])
    dst = str(dftestsample['dstip'].iloc[i])+':'+str(dftestsample['dsport'].iloc[i])
    nodestest.add(src)
    nodestest.add(dst)

# Prepare an adjacency matrix of the nodes
adjacency_matrix_test = np.zeros((len(nodestest), len(nodestest)), dtype=float)
label_matrix_test = np.zeros((len(nodestest), len(nodestest)))

# Add the columns srcip:sport and dstip:dsport to the adjacency matrix
for i in range(len(dftestsample)):
    src = str(dftestsample['srcip'].iloc[i])+':'+str(dftestsample['sport'].iloc[i])
    dst = str(dftestsample['dstip'].iloc[i])+':'+str(dftestsample['dsport'].iloc[i])
    src_index = list(nodestest).index(src)
    dst_index = list(nodestest).index(dst)

    if dftestsample['Label'].iloc[i] == 1:
        if label_matrix_test[src_index,dst_index] == 0:
            adjacency_matrix_test[src_index, dst_index] = dftestsample['sttl'].iloc[i]
            label_matrix_test[src_index, dst_index] = 1  

    else:
        adjacency_matrix_test[src_index, dst_index] = dftestsample['sttl'].iloc[i]


# Flatten tuple values in the adjacency matrix
adjacency_matrix_test_flat = adjacency_matrix_test.reshape(-1, 1)
label_test_flat = label_matrix_test.reshape(-1, 1)

In [17]:
len(nodestest)

732

In [18]:
len(dftestsample)

2536

Model Evaluation

In [22]:
predictions = model.predict(adjacency_matrix_test_flat)
predictions = (predictions > 0.5).astype(int)



14792/14792 [==============================] - 72s 5ms/step


In [23]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(label_test_flat, predictions)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    473252
         1.0       0.76      0.99      0.86        92

    accuracy                           1.00    473344
   macro avg       0.88      0.99      0.93    473344
weighted avg       1.00      1.00      1.00    473344



2. LSTM for STTL & CT_STATE_TTL

In [ ]:
#Sample 1% of the dataset
dfsample = df.sample(frac=0.01, random_state=42)

nodes = set()
for i in range(len(dfsample)):
    src = str(dfsample['srcip'].iloc[i])+':'+str(dfsample['sport'].iloc[i])
    dst = str(dfsample['dstip'].iloc[i])+':'+str(dfsample['dsport'].iloc[i])
    nodes.add(src)
    nodes.add(dst)

# Ignore (Was supposed to be for GCN)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import f1_score

# Define GCN layer
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support) + self.bias
        return output

# Define GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(input_dim, hidden_dim)
        self.gc2 = GraphConvolution(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, adj):
        h = self.relu(self.gc1(x, adj))
        out = self.gc2(h, adj)
        return out

# Function to calculate F1 score
def calculate_f1_score(labels, predictions):
    f1_score_0 = f1_score(labels, predictions, pos_label=0)
    f1_score_1 = f1_score(labels, predictions, pos_label=1)
    return f1_score_0, f1_score_1

# Prepare data (sample data)
# Replace this part with your actual data loading and preprocessing

# Assuming adjacency matrix is already prepared
adjacency_matrix = torch.tensor([[0, 1, 1],
                                 [1, 0, 1],
                                 [1, 1, 0]], dtype=torch.float32)

# Assuming feature matrix is already prepared
feature_matrix = torch.tensor([[0.1, 0.2],
                               [0.3, 0.4],
                               [0.5, 0.6]], dtype=torch.float32)

# Assuming labels are already prepared
labels = torch.tensor([0, 1, 0], dtype=torch.long)

# Hyperparameters
input_dim = feature_matrix.shape[1]
hidden_dim = 16
output_dim = 2
lr = 0.01
epochs = 100

# Initialize model, optimizer, and loss function
model = GCN(input_dim, hidden_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(feature_matrix, adjacency_matrix)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(feature_matrix, adjacency_matrix).argmax(dim=1)
    f1_score_0, f1_score_1 = calculate_f1_score(labels.numpy(), predictions.numpy())
    print(f'F1 Score for Label=0: {f1_score_0}')
    print(f'F1 Score for Label=1: {f1_score_1}')